# Create a Monolayer

Create a Monolayer of the original structure. 

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Materials (in the outer runtime) before running the notebook.
1. Set notebook parameters in cell 1.1. below (or use the default values).
1. Click “Run” > “Run All” to run all cells. 
1. Scroll down to view results. 

## 1. Prepare the Environment
### 1.1. Set up transformation parameters 

In [ ]:
XY_SUPERCELL_MATRIX = [
    [3, 0],
    [0, 3],
]

VACUUM = 10.0  # Angstrom

### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages

    await install_packages("")

### 1.3. Get input materials

In [ ]:
from utils.jupyterlite import get_materials

materials = get_materials(globals())
material = materials[0]

### 1.4. Preview the material

In [ ]:
from utils.visualize import visualize_materials as visualize

visualize(material, repetitions=[3, 3, 1], rotation="0x")
visualize(material, repetitions=[3, 3, 1], rotation="-90x")

## 2. Create Monolayer

### 2.1. Create slab configuration
Slab Configuration lets define the slab thickness, vacuum, and the Miller indices of the interfacial plane and get the slabs with possible terminations.


In [ ]:
from mat3ra.made.tools.build.slab import SlabConfiguration

slab_configuration = SlabConfiguration(
    bulk=material,
    miller_indices=(0, 0, 1),
    thickness=1,
    vacuum=VACUUM,
    xy_supercell_matrix=XY_SUPERCELL_MATRIX,
    use_orthogonal_z=True,
    use_conventional_cell=True,
)

### 2.2 Create the slab of Single Layer

In [ ]:
from mat3ra.made.tools.modify import translate_to_z_level, filter_by_box
from mat3ra.made.tools.build.slab import create_slab

slab = create_slab(slab_configuration)

# Get only the top layer of two -- for AB stacked bilayers forming a 3D bulk basis
if any(substring in material.name for substring in ["3D", "Bulk"]):
    centered_slab = translate_to_z_level(slab, z_level="center")
    half_filtered_slab = filter_by_box(centered_slab, [0, 0, 0.5], [1, 1, 1])
    monolayer = translate_to_z_level(half_filtered_slab, z_level="center")

## 3. Visualize the result

In [ ]:
visualize(monolayer, repetitions=[1, 1, 1], rotation="0x")
visualize(monolayer, repetitions=[1, 1, 1], rotation="-90x")

# 4. Pass material to the outside runtime

In [ ]:
from utils.jupyterlite import set_materials

set_materials(monolayer)